In [1]:
import sys
sys.path.append("..")

In [15]:
import sys
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from model import JoeyLLM
from data import get_dataloader
from utils.logger import wandbLogger
from train.trainer import  Trainer

In [3]:
# Go UP one level to find the conf directory
with hydra.initialize(config_path="../configs", version_base=None):
    cfg = hydra.compose(config_name="config")

In [4]:

print("✅ Loaded Config:")

wandbLogger.set_mode(cfg.wandb.mode)

logger = wandbLogger(
    project_name=cfg.wandb.project,
    config=OmegaConf.to_container(cfg, resolve=True)
)


✅ Loaded Config:


In [5]:

print("📦 Loading Dataset...")
dataloader = get_dataloader(
    data_path=cfg.data.data_path,
    chunk_size=cfg.data.chunk_size,
    buffer_text_size=cfg.data.buffer_text_size,
    batch_size=cfg.data.batch_size,
    num_workers=cfg.data.num_workers
)
    
    

📦 Loading Dataset...


/home/remote/u1138167/JoeyLLM/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# Testing dataloader
for batch in dataloader:
    print("✅ Got batch with shape:", batch.shape)
    break


✅ Got batch with shape: torch.Size([32, 512])


In [12]:
print("🧠 Initializing Model...")
model = JoeyLLM(
    vocab_size=cfg.model.vocab_size,
    max_seq_len=cfg.model.max_seq_len,
    embed_dim=cfg.model.embed_dim,
    num_layers=cfg.model.num_layers,
    num_heads=cfg.model.num_heads,
    dropout=cfg.model.dropout,
)
logger.watch_model(model, log="all", log_freq=10)

🧠 Initializing Model...


In [13]:
    
print("📈 Loading Optimizer")
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), weight_decay=0.1)


📈 Loading Optimizer


In [16]:

print("🚀 Launching Trainer...")
trainer = Trainer(
    model=model,
    dataloader=dataloader,
    optimizer=optimizer,
    scheduler=None,
    logger=None,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

🚀 Launching Trainer...


In [ ]:
trainer.fit(
    num_epochs=1,
    checkpoint_path="checkpoints/checkpoint.pth"
)

In [ ]:
logger.finish()
print("✅ Training Done!")   